In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
from pathlib import Path

from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [3]:
# Load the data
file_path = "https://raw.githubusercontent.com/kwinterling/FinalProject/main/smoker_data/Resources/train_dataset.csv"
train_df = pd.read_csv(file_path)

# Drop the null columns where all values are null
train_df = train_df.dropna(axis='columns', how='all')

# Drop the null rows
train_df = train_df.dropna()


train_df.head()

,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),systolic,relaxation,...,HDL,LDL,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,dental caries,smoking
0,35,170,85,97.0,0.9,0.9,1,1,118,78,...,70,142,19.8,1,1.0,61,115,125,1,1
1,20,175,110,110.0,0.7,0.9,1,1,119,79,...,71,114,15.9,1,1.1,19,25,30,1,0
2,45,155,65,86.0,0.9,0.9,1,1,110,80,...,57,112,13.7,3,0.6,1090,1400,276,0,0
3,45,165,80,94.0,0.8,0.7,1,1,158,88,...,46,91,16.9,1,0.9,32,36,36,0,0
4,20,165,60,81.0,1.5,0.1,1,1,109,64,...,47,92,14.9,1,1.2,26,28,15,0,0


In [4]:
test_file_path = "https://raw.githubusercontent.com/kwinterling/FinalProject/main/smoker_data/Resources/test_dataset.csv"

test_df = pd.read_csv(file_path)

test_df.head()

,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),systolic,relaxation,...,HDL,LDL,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,dental caries,smoking
0,35,170,85,97.0,0.9,0.9,1,1,118,78,...,70,142,19.8,1,1.0,61,115,125,1,1
1,20,175,110,110.0,0.7,0.9,1,1,119,79,...,71,114,15.9,1,1.1,19,25,30,1,0
2,45,155,65,86.0,0.9,0.9,1,1,110,80,...,57,112,13.7,3,0.6,1090,1400,276,0,0
3,45,165,80,94.0,0.8,0.7,1,1,158,88,...,46,91,16.9,1,0.9,32,36,36,0,0
4,20,165,60,81.0,1.5,0.1,1,1,109,64,...,47,92,14.9,1,1.2,26,28,15,0,0


In [5]:
df = pd.concat([train_df, test_df], axis=0, join="outer")
df

non_smokers = df[df.smoking==0]
smokers = df[df.smoking==1]
new_df = pd.concat([non_smokers,smokers])
new_df

,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),systolic,relaxation,...,HDL,LDL,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,dental caries,smoking
1,20,175,110,110.0,0.7,0.9,1,1,119,79,...,71,114,15.9,1,1.1,19,25,30,1,0
2,45,155,65,86.0,0.9,0.9,1,1,110,80,...,57,112,13.7,3,0.6,1090,1400,276,0,0
3,45,165,80,94.0,0.8,0.7,1,1,158,88,...,46,91,16.9,1,0.9,32,36,36,0,0
4,20,165,60,81.0,1.5,0.1,1,1,109,64,...,47,92,14.9,1,1.2,26,28,15,0,0
6,40,175,90,95.0,0.9,1.0,1,1,130,88,...,39,102,16.5,1,1.0,19,22,19,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38975,30,180,75,85.0,1.5,1.2,1,1,123,71,...,67,107,16.2,1,0.8,23,24,33,0,1
38978,40,170,65,77.0,1.5,1.5,1,1,110,62,...,79,91,16.1,1,0.9,28,43,36,1,1
38981,40,170,105,124.0,0.6,0.5,1,1,141,85,...,48,138,17.1,1,0.8,24,23,35,1,1
38982,40,160,55,75.0,1.5,1.5,1,1,95,69,...,79,116,12.0,1,0.6,24,20,17,0,1


In [6]:
df.dtypes

age                      int64
height(cm)               int64
weight(kg)               int64
waist(cm)              float64
eyesight(left)         float64
eyesight(right)        float64
hearing(left)            int64
hearing(right)           int64
systolic                 int64
relaxation               int64
fasting blood sugar      int64
Cholesterol              int64
triglyceride             int64
HDL                      int64
LDL                      int64
hemoglobin             float64
Urine protein            int64
serum creatinine       float64
AST                      int64
ALT                      int64
Gtp                      int64
dental caries            int64
smoking                  int64
dtype: object

In [7]:
# Drop the non-beneficial ID columns.
# new_df = df.drop(['waist(cm)', 'height(cm)', 'weight(kg)', 'eyesight(left)', 'eyesight(right)', 'hearing(left)', 'hearing(right)', 'Urine protein', 'dental caries', 'relaxation', 'AST', 'age', 'systolic'], axis=1)
df = df[(df["age"] < 40) | (df["age"] > 45)]
new_df = df.drop(['eyesight(left)', 'eyesight(right)', 'hearing(left)', 'hearing(right)', 'Urine protein', 'dental caries', 'relaxation', 'AST', 'age', 'systolic'], axis=1)
# new_df = df
new_df.head()

,height(cm),weight(kg),waist(cm),fasting blood sugar,Cholesterol,triglyceride,HDL,LDL,hemoglobin,serum creatinine,ALT,Gtp,smoking
0,170,85,97.0,97,239,153,70,142,19.8,1.0,115,125,1
1,175,110,110.0,88,211,128,71,114,15.9,1.1,25,30,0
4,165,60,81.0,100,179,200,47,92,14.9,1.2,28,15,0
5,160,50,78.0,114,177,74,98,64,13.9,1.0,23,70,1
11,155,50,72.0,83,135,35,59,69,12.5,0.9,12,11,0


In [8]:
X = new_df.drop(columns="smoking")
y = pd.DataFrame(new_df["smoking"])

In [9]:
X.describe()

,height(cm),weight(kg),waist(cm),fasting blood sugar,Cholesterol,triglyceride,HDL,LDL,hemoglobin,serum creatinine,ALT,Gtp
count,46742.000000,46742.000000,46742.000000,46742.000000,46742.000000,46742.000000,46742.000000,46742.000000,46742.000000,46742.000000,46742.000000,46742.000000
mean,165.054555,66.797955,83.108964,100.080014,195.551538,127.303924,56.354499,114.542467,14.854033,0.906769,28.280818,41.023405
std,9.586352,12.784073,8.842484,20.622048,37.318825,69.790680,14.292623,43.554985,1.371719,0.224226,33.485914,51.611985
min,130.000000,30.000000,51.000000,46.000000,77.000000,8.000000,4.000000,1.000000,6.300000,0.100000,1.000000,2.000000
25%,160.000000,60.000000,77.000000,89.000000,170.000000,76.000000,46.000000,90.000000,13.900000,0.800000,16.000000,18.000000
50%,165.000000,65.000000,83.000000,96.000000,193.000000,110.000000,55.000000,112.000000,15.000000,0.900000,22.000000,27.000000
75%,170.000000,75.000000,88.900000,105.000000,219.000000,160.000000,64.000000,136.000000,15.800000,1.000000,32.000000,44.000000
max,190.000000,135.000000,129.000000,398.000000,445.000000,466.000000,359.000000,1860.000000,20.900000,10.000000,2914.000000,999.000000


In [10]:
y['smoking'].value_counts()

0    29338
1    17404
Name: smoking, dtype: int64

In [11]:
from collections import Counter
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1)

scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
X_train.shape

(35056, 12)

In [12]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(35056, 12)
(11686, 12)
(35056, 1)
(11686, 1)


In [13]:
Counter(y_train)

Counter({'smoking': 1})

In [14]:
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.datasets import make_classification

In [19]:
from imblearn.ensemble import BalancedRandomForestClassifier
brf = BalancedRandomForestClassifier(n_estimators=100, max_depth=32, random_state=1)
brf

BalancedRandomForestClassifier(max_depth=32, random_state=1)

In [20]:
brf.fit(X_train_scaled, y_train)
y_pred = brf.predict(X_test_scaled)

<ipython-input-20-aff86d4fa7d9>:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  brf.fit(X_train_scaled, y_train)


In [17]:
rf = BalancedRandomForestClassifier()
parameters = {
    "n_estimators": [5, 20, 40, 80, 100, 200],
    "max_depth": [2, 4, 8, 16, 32, None]
}

cv = GridSearchCV(rf, parameters, cv=5)
cv.fit(X_train.values, y_train.values.ravel())

GridSearchCV(cv=5, estimator=BalancedRandomForestClassifier(),
             param_grid={'max_depth': [2, 4, 8, 16, 32, None],
                         'n_estimators': [5, 20, 40, 80, 100, 200]})

In [18]:
print(cv.best_params_)

{'max_depth': 32, 'n_estimators': 100}


In [21]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = brf.predict(X_test_scaled)
y_pred

array([0, 1, 1, ..., 1, 1, 1])

In [22]:
balanced_accuracy_score(y_test, y_pred)

0.9433254731463656

In [23]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
y_pred = brf.predict(X_test_scaled)
confusion_matrix(y_test, y_pred)

array([[6875,  528],
       [ 180, 4103]])

In [24]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.97      0.93      0.96      0.95      0.94      0.89      7403
          1       0.89      0.96      0.93      0.92      0.94      0.89      4283

avg / total       0.94      0.94      0.95      0.94      0.94      0.89     11686



In [25]:
# List the features sorted in descending order by feature importance
importances = brf.feature_importances_
importances
sorted(zip(brf.feature_importances_, X.columns), reverse=True)

[(0.13111200597660638, 'hemoglobin'),
 (0.1228443776370219, 'height(cm)'),
 (0.11236640905354289, 'Gtp'),
 (0.08752896767142385, 'triglyceride'),
 (0.07682939207596311, 'LDL'),
 (0.07634596750207283, 'Cholesterol'),
 (0.07324497771776307, 'ALT'),
 (0.07285111278670323, 'HDL'),
 (0.07276033156291774, 'waist(cm)'),
 (0.07246252387381023, 'fasting blood sugar'),
 (0.05187806314448923, 'serum creatinine'),
 (0.049775870997685524, 'weight(kg)')]

In [26]:
from sklearn.ensemble import GradientBoostingClassifier

xgb = GradientBoostingClassifier()

xgb.fit(X_train_scaled, y_train.values.ravel())
y_pd = xgb.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pd)


0.7039954900880243